In [18]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [19]:
dataset_path = "/content/drive/MyDrive/7-7 dataset"

In [20]:
img_data_generator = ImageDataGenerator(
    rescale=1./255 ,
    validation_split=0.1,
    horizontal_flip=True
)
train_data= img_data_generator.flow_from_directory(
    dataset_path ,
    shuffle=True,
    subset="training",
    target_size=(224,224)
)

validation_data = img_data_generator.flow_from_directory(
    dataset_path,
    shuffle=True,
    subset="validation",
    target_size=(224,224)
)

Found 1227 images belonging to 14 classes.
Found 131 images belonging to 14 classes.


In [21]:
original_model = tf.keras.applications.VGG16(weights="imagenet" , include_top=True) # az vazn haye train shode imagenet istefade mikone
# agar weights="None" bashe , memari hamoon VGG16 hast VALI hameye vazn ha RANDDOM mishan
# include_top= لایه های نهایی شبکه یا همان فولی کانکتدها
original_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## لایه آخر را باید حذف کرد و به جای ۱۰۰۰ تا نرون ، ۱۴ تا گذاشت


In [22]:
# لایه های فولی کانکتد را حذف میکنیم
model = tf.keras.applications.VGG16(weights="imagenet" , include_top=False) # az vazn haye train shode imagenet istefade mikone
model.summary()
# در پایین می بینیم که اندازه ها نان شده اند
# راه حل : ابعاد ورودی را خودمون باید تعریف کنیم حالا

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [23]:
model = tf.keras.applications.VGG16(weights="imagenet" , include_top=False , input_shape=(224,224,3)) # az vazn haye train shode imagenet istefade mikone
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Adding fully connected layers to model :

In [24]:
final_model = tf.keras.models.Sequential([
    model,

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000 , activation="relu"),
    tf.keras.layers.Dense(2000 , activation="relu"),
    tf.keras.layers.Dense(14 , activation="softmax")

])
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 1000)              25089000  
                                                                 
 dense_4 (Dense)             (None, 2000)              2002000   
                                                                 
 dense_5 (Dense)             (None, 14)                28014     
                                                                 
Total params: 41833702 (159.58 MB)
Trainable params: 41833702 (159.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001) ,
              loss=tf.keras.losses.categorical_crossentropy ,
              metrics=["accuracy"] )

In [ ]:
history = final_model.fit(train_data , validation_data=validation_data , epochs=30)


Epoch 1/30
18/39 [============>.................] - ETA: 23:49 - loss: 4.6572 - accuracy: 0.0642